In [23]:
import random
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np 
import matplotlib as plt
torch.manual_seed(1)

In [24]:
def move(i,j,n,direction,step,t,v):
    if direction==0 and j+step<=n-1:#right
        for k in range(0,step):
            j=j+1
            t[i][j]=v 
    elif direction==1 and j-step>=0:#left
        for k in range(0,step):
            j=j-1
            t[i][j]=v
    elif direction==2 and i-step>=0:#up
        for k in range(0,step):
            i=i-1
            t[i][j]=v
    elif direction==3 and i+step<=n-1:#down
        for k in range(0,step):
            i=i+1
            t[i][j]=v
    return i,j
    
def inrange(i,j,direction,t):
    if direction==0 and (t[i+1][j]==0 or t[i][j+1]==0 or t[i-1][j]==0):
        return True
    elif direction==1 and (t[i+1][j]==0 or t[i][j-1]==0 or t[i-1][j]==0):
        return True
    elif direction==2 and (t[i][j-1]==0 or t[i+1][j]==0 or t[i][j+1]==0):
        return True
    elif direction==3 and (t[i][j-1]==0 or t[i-1][j]==0 or t[i][j+1]==0):
        return True
    return False

In [37]:
def generate_Solution(i,j,n,solution):    
    #----------------------------------------
    #Create the Solution First: 1 x n x n
    #----------------------------------------
    solution[i][j]=0
    while (i!=0 or j!=n-1):
        direction=random.choice([0,2])
        step=random.choice([1,1,2,3])
        i,j=move(i,j,n,direction,step,solution,0)
    return solution

def generate_Problem(i,j,n,problem):
    #---------------------------------------------------
    #Create the Problem Based on Solution: 1 x n x n
    #---------------------------------------------------
    branchcount=2#branchcount=random.randint(5,6)
    branchcoords=random.sample(range(0, n-1,2),branchcount) #branchcoords=random.sample(range(5, n-5,2),branchcount)
    for coord in branchcoords:
        for y in range(n-1):
            if problem[y][coord]==0:
                i=y
                j=coord
                alternative=random.randint(0,1)#left up if 0, right down if 1
                #start drawing line
                while ((0<i and i<n-1 and j!=0) or (0<j and j<n-1 and i!=0)):
                    if alternative==1:
                        direction=random.choice([1,2])
                    else:
                        direction=random.choice([0,3])
                    step=random.choice([1,1,2,3])
                    i,j=move(i,j,n,direction,step,problem,0)
                break
    for coords in branchcoords:
        for x in range(n-1):
            if problem[coord][x]==0:
                i=coord
                j=x
                alternative=random.randint(0,1)#left up if 0, right down if 1
                #start drawing line
                while ((0<i and i<n-1 and j!=0) or (0<j and j<n-1 and i!=0) and (not inrange(i,j,direction,problem))):
                    if alternative==1:
                        direction=random.choice([0,2])
                    else:
                        direction=random.choice([1,3])
                    step=random.choice([1,1,2,3])
                    i,j=move(i,j,n,direction,step,problem,0)
                break
    return problem

test


In [86]:
n=32
m=3
mazes = torch.zeros(m,1,n,n)
sols = torch.zeros(m,1,n,n)
for k in range(m):
    solution = torch.ones(n,n)
    problem = torch.ones(n,n)
    i,j=n-1,0
    solution=generate_Solution(i,j,n,solution)
    i,j=n-1,0
    problem=generate_Problem(i,j,n,solution.clone())

    #print(solution.size())
    print(problem.size())

    print(problem)
    mazes[k,0]=problem
    sols[k,0]=solution
    print(mazes[k,0])
    print(sols[k,0])

    transform=T.ToPILImage()
    img=transform(mazes[k,0])
    img.show(mazes[k,0])
dataset = torch.utils.data.TensorDataset(mazes, sols)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=0)


torch.Size([32, 32])
tensor([[1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        ...,
        [0., 1., 1.,  ..., 1., 0., 1.],
        [0., 1., 1.,  ..., 1., 0., 0.],
        [0., 1., 1.,  ..., 1., 1., 0.]])
tensor([[1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        ...,
        [0., 1., 1.,  ..., 1., 0., 1.],
        [0., 1., 1.,  ..., 1., 0., 0.],
        [0., 1., 1.,  ..., 1., 1., 0.]])
tensor([[1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        ...,
        [0., 1., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 1., 1., 1.]])
torch.Size([32, 32])
tensor([[1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        [1., 1., 1.,  ..., 1., 1., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  .